# import packages

In [12]:
import numpy as np
import pandas as pd
import scipy as sp
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import xgboost as xgb
import catboost as cb
import lightgbm as lgb

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)


# import data

In [14]:
data_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/Mobility_data/applemobilitytrends_2020_06_29.xlsx'
mobility_data = pd.read_excel(data_path)
county_data = mobility_data[mobility_data['county'].str.contains('County')].reset_index(drop = True)
county_data

,geo_type,county,transportation_type,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29
0,county,Alachua County,driving,Florida,United States,100,102.21,111.36,116.55,141.97,130.21,92.69,104.06,101.20,103.93,106.16,131.05,120.30,91.92,97.30,102.00,99.21,110.02,144.46,129.98,93.15,97.86,100.93,105.94,108.80,135.37,128.78,99.12,100.18,105.30,108.33,127.00,157.51,143.37,108.81,107.82,107.65,110.22,116.62,140.65,135.36,102.54,100.90,103.86,119.22,124.22,157.90,117.31,86.23,89.77,89.19,91.26,94.83,114.68,119.74,99.42,108.03,110.97,113.35,119.84,140.00,119.29,86.35,91.64,85.15,79.61,76.55,81.44,67.81,50.88,55.82,50.67,52.95,52.62,57.60,52.28,41.10,49.14,49.81,51.99,51.90,53.91,45.36,34.70,46.15,47.72,47.88,50.62,56.55,50.89,33.68,50.68,48.61,51.46,56.66,63.26,54.34,47.48,54.18,57.14,59.33,61.13,64.23,62.90,49.31,60.22,62.65,65.62,66.78,85.60,72.56,59.45,68.54,73.79,73.60,76.47,86.91,76.87,55.54,NaN,NaN,80.97,85.03,98.57,91.66,76.43,79.95,86.47,88.27,97.84,110.89,100.32,81.67,75.80,88.85,91.17,94.49,111.02,106.77,86.88,92.84,92.63,92.19,96.90,117.39,106.39,79.76,95.01,99.14,101.95,103.45,123.29,123.28,95.84,101.77,102.80,104.84,109.38,130.16,121.64,87.92,105.58,104.22,107.49,115.48,127.04,122.16,95.71,104.81
1,county,Baker County,driving,Florida,United States,100,106.73,104.79,115.21,160.07,136.27,107.89,124.29,100.76,102.84,109.70,133.63,136.93,101.12,102.71,99.64,103.96,111.82,147.92,132.97,99.77,112.41,101.98,107.03,113.33,147.85,122.24,115.41,110.07,109.54,116.57,134.95,175.64,166.24,131.82,127.52,114.72,117.29,126.34,156.53,155.38,120.43,114.72,111.98,111.02,156.80,166.07,160.59,132.54,116.30,113.60,115.38,133.93,178.05,172.94,139.74,129.83,126.47,132.34,154.75,197.72,176.77,140.20,132.57,109.57,114.98,117.49,128.91,114.88,97.23,92.57,86.37,88.88,91.75,108.25,99.60,79.83,98.91,83.86,76.30,90.56,87.62,65.68,57.95,76.53,74.52,68.45,79.50,89.21,86.17,58.02,73.89,69.80,73.76,88.84,97.26,89.74,78.98,84.32,88.68,94.75,97.59,104.79,97.19,89.70,99.27,97.00,97.23,100.30,126.20,127.52,104.59,114.06,111.02,112.18,125.68,147.46,127.26,101.22,NaN,NaN,126.67,128.28,160.17,166.83,138.38,128.22,134.36,125.15,157.03,201.16,174.29,144.36,154.62,136.47,126

# try plot one county

In [16]:
county = 'Alachua County'
alachua_county = county_data.query('county == @county').drop(columns = ['geo_type','sub-region','country'],axis = 1)
alachua_county

,county,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29
0,Alachua County,driving,100,102.21,111.36,116.55,141.97,130.21,92.69,104.06,101.2,103.93,106.16,131.05,120.3,91.92,97.3,102.0,99.21,110.02,144.46,129.98,93.15,97.86,100.93,105.94,108.8,135.37,128.78,99.12,100.18,105.3,108.33,127.0,157.51,143.37,108.81,107.82,107.65,110.22,116.62,140.65,135.36,102.54,100.9,103.86,119.22,124.22,157.9,117.31,86.23,89.77,89.19,91.26,94.83,114.68,119.74,99.42,108.03,110.97,113.35,119.84,140.0,119.29,86.35,91.64,85.15,79.61,76.55,81.44,67.81,50.88,55.82,50.67,52.95,52.62,57.6,52.28,41.1,49.14,49.81,51.99,51.9,53.91,45.36,34.7,46.15,47.72,47.88,50.62,56.55,50.89,33.68,50.68,48.61,51.46,56.66,63.26,54.34,47.48,54.18,57.14,59.33,61.13,64.23,62.9,49.31,60.22,62.65,65.62,66.78,85.6,72.56,59.45,68.54,73.79,73.6,76.47,86.91,76.87,55.54,NaN,NaN,80.97,85.03,98.57,91.66,76.43,79.95,86.47,88.27,97.84,110.89,100.32,81.67,75.8,88.85,91.17,94.49,111.02,106.77,86.88,92.84,92.63,92.19,96.9,117.39,106.39,79.76,95.01,99.14,101.95,103.45,123.29,123.28,95.84,101.77,102.8,104.84,109.38,130.16,121.64,87.92,105.58,104.22,107.49,115.48,127.04,122.16,95.71,104.81


In [22]:
melt_alachua = pd.melt(alachua_county, id_vars = ['county','transportation_type'])
melt_alachua['variable'] = pd.to_datetime(melt_alachua['variable'])
melt_alachua['value'] = melt_alachua['value'].apply(lambda x: x-100)
melt_alachua

,county,transportation_type,variable,value
0,Alachua County,driving,2020-01-13,0.00
1,Alachua County,driving,2020-01-14,2.21
2,Alachua County,driving,2020-01-15,11.36
3,Alachua County,driving,2020-01-16,16.55
4,Alachua County,driving,2020-01-17,41.97
5,Alachua County,driving,2020-01-18,30.21
6,Alachua County,driving,2020-01-19,-7.31
7,Alachua County,driving,2020-01-20,4.06
8,Alachua County,driving,2020-01-21,1.20
9,Alachua County,driving,2020-01-22,3.93


In [28]:
fig = px.line(melt_alachua, x="variable", y="value", color='transportation_type', 
              title="Alachua county driving mobility")

fig.show()